In [1]:
from itertools import combinations
from bs4 import BeautifulSoup
from urllib.request import urlopen
import lxml
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from collections import defaultdict

from contextlib import contextmanager

import re
import pandas as pd
from matplotlib import pyplot as plt

from thefuzz import process
from thefuzz import fuzz

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
)

In [17]:
def start_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

#### Info about Zurich districts

In [2]:
# save the url of the website
zurich_districts_url = "https://www.zuerich.com/en/visit/about-zurich/zurichs-districts"

# get the html content of the website
zurich_response = urlopen(zurich_districts_url)
zurich_html_content = zurich_response.read()

In [3]:
# parse the html content
zurich_soup = BeautifulSoup(zurich_html_content, "lxml")

In [4]:
# select all elements with id starting with 's-' and a number between 1 and 12
pattern = re.compile(r"s-[1-9]|s-1[0-2]")
elements = zurich_soup.find_all(id=pattern)

In [5]:
# create a dataframe with the information of the districts
districts = {element.find("h2").text: element.find("p").text for element in elements}
districts_df = pd.DataFrame.from_dict(districts, orient="index", columns=["desc"])


# make the index into a column and split it into district number and district name
districts_df = districts_df.reset_index()
districts_df = (
    districts_df["index"]
    .str.split("–", expand=True)
    .rename({0: "district_number", 1: "district_name"}, axis=1)
    .join(districts_df)
    .drop("index", axis=1)
)
# strip the whitespace from the columns
districts_df["district_number"] = districts_df["district_number"].str.strip()
# create regex to get the number from the district_number column
regex_pattern = re.compile(r"([\d]+)")

# create a new column with the district number
districts_df["district"] = (
    districts_df["district_number"]
    .str.extract(
        regex_pattern,
    )
    .astype("category")
)
districts_df.drop("district_number", axis=1, inplace=True)

districts_df["district_name"] = districts_df["district_name"].str.strip()
districts_df["desc"] = districts_df["desc"].str.strip()

# Add column for the length of the desc
districts_df["desc_length"] = districts_df["desc"].str.len()

print(districts_df)

                      district_name  \
0             Old Town, City Center   
1            Left Shore of the Lake   
2      Creative Quarter of Wiedikon   
3   Creative Quarter of Langstrasse   
4   Creative Quarter of Zürich-West   
5                University Quarter   
6                 On the Zürichberg   
7           Right Shore of the Lake   
8      At the Foot of the Uetliberg   
9               Right of the Limmat   
10                      Zürich Nord   
11                   Schwamendingen   

                                                 desc district  desc_length  
0   The most central district encompasses the hist...        1          355  
1   The architecturally attractive Enge Train Stat...        2          206  
2   Once mainly a working-class neighborhood, the ...        3          215  
3   The district around Langstrasse was long regar...        4          199  
4   In the quarter where huge machines once clatte...        5          287  
5   University buildings

In [6]:
districts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   district_name  12 non-null     object  
 1   desc           12 non-null     object  
 2   district       12 non-null     category
 3   desc_length    12 non-null     int64   
dtypes: category(1), int64(1), object(2)
memory usage: 824.0+ bytes


In [7]:
# create a styler object and set the wrap parameter to True
styler = districts_df.style.set_properties(**{"white-space": "pre-wrap"})

formatted_df = styler.format({"description": lambda x: x})
formatted_df

,district_name,desc,district,desc_length
0,"Old Town, City Center","The most central district encompasses the historical Old Town on both banks of the River Limmat, as well as the area to the south bordering on the lake basin. It is home to the prestigious Bahnhofstrasse, magnificent guild houses, imposing churches, and the famous Opera House. The best view of District 1 is to be had from the top of the Karlsturm tower.",1,355
1,Left Shore of the Lake,"The architecturally attractive Enge Train Station built out of Ticino granite, the Museum Rietberg with its beautiful park, the Seebad Enge lido, and the Rote Fabrik give this district a Mediterranean feel.",2,206
2,Creative Quarter of Wiedikon,"Once mainly a working-class neighborhood, the district below the Uetliberg is now a popular residential area with cozy cafés, boutiques, and vintage stores. The Houdini movie theater also lies on Wiedikon territory.",3,215
3,Creative Quarter of Langstrasse,"The district around Langstrasse was long regarded as a den of iniquity. Nowadays, it is known for its diverse restaurants and never-sleeping nightlife, with bars such as the Olé and the Club Zukunft.",4,199
4,Creative Quarter of Zürich-West,"In the quarter where huge machines once clattered away, now nightclubs, cultural institutions, and universities cluster around the Prime Tower. Converted structures such as the Viadukt and designer stores like the Freitag Tower give the neighborhood its characteristic trendy atmosphere.",5,287
5,University Quarter,"University buildings, Jugendstil villas, green areas, and the Dynamo cultural center characterize Zurich’s District 6. Thanks to its tranquility and closeness to the city center, it is a much sought-after residential area.",6,222
6,On the Zürichberg,"The hillside location, little traffic, and fabulous views mean than living here does not come cheap. Neighbors include Zurich Zoo and the luxury hotel, The Dolder Grand.",7,169
7,Right Shore of the Lake,"This district starts behind the Opera House and stretches as far as the open-air bathing facility, Seebad Tiefenbrunnen. It features boutiques, villas, the Chinawiese recreational area, and the Seebad Utoquai outdoor swimming bath.",8,231
8,At the Foot of the Uetliberg,"For a long time, this district was solely known for the Letzigrund Stadium. In recent years, however, the quarter around Altstetten has become THE place to be, with creative club and gastronomy concepts.",9,203
9,Right of the Limmat,"This district boasts a fantastic location: in summer, everyone meets at the Unterer Letten and Oberer Letten riverside lidos to swim and bask in the sun. In addition, the view over Zurich from the Waid quarter is unbeatable.",10,224


In [8]:
districts_df.to_csv("../data/zurich_districts.csv", index=False)

#### Info about Dog breeds from AKC


In [9]:
akc_dog_breed_groups_url = "https://www.akc.org/public-education/resources/general-tips-information/dog-breeds-sorted-groups/"

# get the html content of the website
akc_response = urlopen(akc_dog_breed_groups_url)
akc_html_content = akc_response.read()

In [10]:
# Parse the html content
akc_soup = BeautifulSoup(akc_html_content, "lxml")

In [11]:
# get the elements with links as the text is the info you want
breed_list = []
link_list = []
elements = akc_soup.find_all("a", href=True)
for element in elements:
    if "dog-breeds" in element.get("href"):
        breed_list.append(element.text.strip())
        link_list.append(element.get("href"))

In [12]:
# link_list
# len(link_list)
# len(breed_list)
breed_link_dict = dict(zip(breed_list, link_list))

breed_link_dict_casefolded = {
    key.casefold(): value for key, value in breed_link_dict.items()
}
breed_link_dict_casefolded

{'view all breeds': 'https://www.akc.org/dog-breeds/',
 'dog breeds': '/expert-advice/dog-breeds/',
 'list of breeds by group': 'https://www.akc.org/public-education/resources/general-tips-information/dog-breeds-sorted-groups/',
 'herding group': 'https://www.akc.org/dog-breeds/groups/herding/',
 'australian cattle dog': 'https://www.akc.org/dog-breeds/australian-cattle-dog/',
 'australian shepherd': 'https://www.akc.org/dog-breeds/australian-shepherd/',
 'bearded collie': 'https://www.akc.org/dog-breeds/bearded-collie/',
 'beauceron': 'https://www.akc.org/dog-breeds/beauceron/',
 'belgian laekenois': 'https://www.akc.org/dog-breeds/belgian-laekenois/',
 'belgian malinois': 'https://www.akc.org/dog-breeds/belgian-malinois/',
 'belgian sheepdog': 'https://www.akc.org/dog-breeds/belgian-sheepdog/',
 'belgian tervuren': 'https://www.akc.org/dog-breeds/belgian-tervuren/',
 'bergamasco': 'https://www.akc.org/dog-breeds/bergamasco-sheepdog/',
 'berger picard': 'https://www.akc.org/dog-breeds

In [13]:
# loop over the breed_list and create a dictionary of group breeds and their sub-breeds
group_breeds = {}
current_group = None
for breed in breed_link_dict_casefolded:
    if re.search(r"group|stock|class", breed):
        current_group = breed
        group_breeds[current_group] = []
    elif current_group is not None:
        group_breeds[current_group].append(breed)

# print the resulting dictionary of breed groups and their breeds
print(group_breeds)

{'list of breeds by group': [], 'herding group': ['australian cattle dog', 'australian shepherd', 'bearded collie', 'beauceron', 'belgian laekenois', 'belgian malinois', 'belgian sheepdog', 'belgian tervuren', 'bergamasco', 'berger picard', 'border collie', 'bouvier des flandres', 'briard', 'canaan dog', 'cardigan welsh corgi', 'collie', 'entlebucher mountain dog', 'finnish lapphund', 'german shepherd dog', 'icelandic sheepdog', 'lancashire heeler', 'miniature american shepherd', 'mudi', 'norwegian buhund', 'old english sheepdog', 'pembroke welsh corgi', 'polish lowland sheepdog', 'puli', 'pumi', 'pyrenean shepherd', 'shetland sheepdog', 'spanish water dog', 'swedish vallhund'], 'hound group': ['afghan hound', 'american english coonhound', 'american foxhound', 'azawakh', 'basenji', 'basset hound', 'beagle', 'black and tan coonhound', 'bloodhound', 'bluetick coonhound', 'borzoi', 'cirneco dell’etna', 'dachshund', 'english foxhound', 'grand basset griffon vendeen', 'greyhound', 'harrier'

In [14]:
# create a dataframe with only 2 columns, one for the breed group and the other for the breeds in that group
# this dataframe would be in long format

group_breeds_df = pd.DataFrame(group_breeds.items(), columns=["breed_group", "breed"])
akc_breeds_df = group_breeds_df.explode("breed")
akc_breeds_df.reset_index(drop=True, inplace=True)
akc_breeds_df["breed_group"] = (
    akc_breeds_df["breed_group"].str.replace(" GROUP", "").str.lower()
)
akc_breeds_df["breed"] = akc_breeds_df["breed"].str.lower()
akc_breeds_df = akc_breeds_df.iloc[1:-1, :]

In [15]:
akc_breeds_df["links"] = akc_breeds_df["breed"].map(breed_link_dict_casefolded)

In [16]:
akc_breeds_df.to_csv("../data/akc_dog_breeds.csv", index=False)

#### Info for Dog breeds from FCI

In [12]:
driver.quit()

In [ ]:

def get_fci_breeds(driver, link):
    """Function to get the breeds from the FCI website.
    It takes as arguments the driver and the link to the page.
    It navigates to each letter and then to each breed and
    gets the translations of the breed name in other languages."""
    name_link_list = []

    driver.get(link)

    # Wait for the letters to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "initiales"))
    )

    letters = driver.find_element(By.CLASS_NAME, "initiales")

    for n, letter in tqdm(enumerate(letters.find_elements(By.TAG_NAME, "a"))):
        try:
            # click first on the letter
            letter.click()
        except StaleElementReferenceException:
            # the element is no longer attached to the DOM so find them again
            letters = driver.find_element(By.CLASS_NAME, "initiales")
            letter = letters.find_elements(By.TAG_NAME, "a")[n]
            letter.click()

        # Wait for the breeds to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "listeraces"))
        )

        breeds = driver.find_element(By.CLASS_NAME, "listeraces")

        breed_text = None
        breed_ref = None
        for n2, breed in tqdm(enumerate(breeds.find_elements(By.TAG_NAME, "a"))):
            try:
                breed_text = breed.text
                breed_ref = breed.get_attribute("href")
                # print(breed_text, breed_ref)
                # click on the breed
                breed.click()
            except StaleElementReferenceException:
                # the element is no longer attached to the DOM so find them again
                breeds = driver.find_element(By.CLASS_NAME, "listeraces")
                breed = breeds.find_elements(By.TAG_NAME, "a")[n2]
                breed_text = breed.text
                breed_ref = breed.get_attribute("href")
                # print(breed_text, breed_ref)
                breed.click()

            # wait for that breed's page to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "racesgridview"))
            )
            table = driver.find_element(By.CLASS_NAME, "racesgridview")
            # get the other languages transations of the breed name
            breed_translations = []
            # get the other languages transations of the breed name
            # only get that column of the table
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows[1:]:
                breed_translations.append(
                    row.find_elements(By.TAG_NAME, "span")[0].text
                )

            # print((breed_text, breed_ref, breed_translations))
            # add the breed, link, and translations to the list
            name_link_list.append((breed_text, breed_ref, breed_translations))
            # go back to the previous page with the breeds
            driver.back()

    driver.quit()

    return name_link_list

In [14]:
# driver = webdriver.Chrome()

fci_nonmenclature_url = "https://fci.be/en/Nomenclature/Default.aspx"


In [18]:
my_d = start_driver()
fci_list = get_fci_breeds(my_d, fci_nonmenclature_url)

0it [00:00, ?it/s]

('AFFENPINSCHER', 'https://fci.be/en/nomenclature/AFFENPINSCHER-186.html', ['AFFENPINSCHER', 'AFFENPINSCHER', 'AFFENPINSCHER', 'AFFENPINSCHER'])


('AFGHAN HOUND', 'https://fci.be/en/nomenclature/AFGHAN-HOUND-228.html', ['AFGHAN HOUND', 'LEVRIER AFGHAN', 'AFGHANISCHER WINDHUND', 'LEBREL AFGANO'])


("AÏDI (CHIEN DE MONTAGNE DE L'ATLAS)", 'https://fci.be/en/nomenclature/ATLAS-MOUNTAIN-DOG-AIDI-247.html', ['ATLAS MOUNTAIN DOG (AIDI)', "CHIEN DE MONTAGNE DE L'ATLAS (AÏDI)", 'ATLAS-BERGHUND (AÏDI)', 'PERRO DE MONTAÑA DEL ATLAS (AÏDI)'])


('AIREDALE TERRIER', 'https://fci.be/en/nomenclature/AIREDALE-TERRIER-7.html', ['AIREDALE TERRIER', 'AIREDALE TERRIER', 'AIREDALE TERRIER', 'AIREDALE TERRIER'])


...

('YUZHNORUSSKAYA OVCHARKA', 'https://fci.be/en/nomenclature/SOUTH-RUSSIAN-SHEPHERD-DOG-326.html', ['SOUTH RUSSIAN SHEPHERD DOG', 'BERGER DE RUSSIE MERIDIONALE', 'SÜDRUSSISCHER OVTCHARKA', 'PERRO DE PASTOR DE RUSIA MERIDIONAL'])


('ZAPADNO-SIBIRSKAÏA LAÏKA', 'https://fci.be/en/nomenclature/WEST-SIBERIAN-LAIKA-306.html', ['WEST SIBERIAN LAIKA', 'LAIKA DE SIBERIE OCCIDENTALE', 'WESTSIBIRISCHER LAÏKA', 'LAIKA DE SIBERIA OCCIDENTAL'])


('ZWERGPINSCHER', 'https://fci.be/en/nomenclature/MINIATURE-PINSCHER-185.html', ['MINIATURE PINSCHER', 'PINSCHER NAIN', 'ZWERGPINSCHER', 'PINSCHER MINIATURA'])


3it [00:02,  1.16it/s]
26it [06:19, 14.60s/it]

('ZWERGSCHNAUZER', 'https://fci.be/en/nomenclature/MINIATURE-SCHNAUZER-183.html', ['MINIATURE SCHNAUZER', 'SCHNAUZER NAIN', 'ZWERGSCHNAUZER', 'SCHNAUZER MINIATURA'])


In [23]:
fci_breeds_trans_df = pd.DataFrame(fci_list, columns=["breed", "link", "translations"])
fci_breeds_trans_df.sample(3)

,breed,link,translations
160,GONCZY POLSKI,https://fci.be/en/nomenclature/POLISH-HUNTING-...,"[POLISH HUNTING DOG, CHIEN COURANT POLONAIS, P..."
294,SCHIPPERKE,https://fci.be/en/nomenclature/SCHIPPERKE-83.html,"[SCHIPPERKE, SCHIPPERKE, SCHIPPERKE, SCHIPPERKE]"
207,KAVKAZSKAÏA OVTCHARKA,https://fci.be/en/nomenclature/CAUCASIAN-SHEPH...,"[CAUCASIAN SHEPHERD DOG, BERGER DU CAUCASE, KA..."


In [25]:
fci_breeds_trans_df[fci_breeds_trans_df["breed"].str.startswith("V")]

,breed,link,translations
337,VÄSTGÖTASPETS,https://fci.be/en/nomenclature/SWEDISH-VALLHUN...,"[SWEDISH VALLHUND, VALLHUND SUEDOIS - SPITZ DE..."
338,VOLPINO ITALIANO,https://fci.be/en/nomenclature/ITALIAN-VOLPINO...,"[ITALIAN VOLPINO, VOLPINO ITALIEN, ITALIENISCH..."
339,VOSTOTCHNO-SIBIRSKAÏA LAÏKA,https://fci.be/en/nomenclature/EAST-SIBERIAN-L...,"[EAST SIBERIAN LAIKA, LAIKA DE SIBERIE ORIENTA..."


In [24]:
fci_breeds_trans_df.to_csv("../data/fci_breeds_trans.csv", index=False)

In [17]:
from lxml import etree

fci_url = "https://www.fci.be/en/Nomenclature/educationGroupe.aspx"
fci_response = urlopen(fci_url)
fci_html_content = fci_response.read()

fci_parsed_html = etree.HTML(fci_html_content)

In [18]:
breed_groups = {}
elements = fci_parsed_html.xpath("//*[@class='nom']")
for element in elements:
    breed_groups[element.text] = element.get("href")

In [19]:
fci_breeds_df = (
    pd.DataFrame.from_dict(breed_groups, orient="index", columns=["link"])
    .reset_index()
    .rename(columns={"index": "breed"})
)

# define regex pattern to get what is in the most right brackets
regex_pattern = re.compile(r"\((?=[^()]*\))([^()]+)\)$")


# fci_breeds_df["breed"].str.extract(regex_pattern)
# fci_breeds_df["breed"].str.split("(", n=1, expand=True).rename(
#     columns={0: "breed_orig", 1: "breed_en"}
# )
#
fci_breeds_df.sample(3)

,breed,link
2,CANAAN DOG (273),/en/nomenclature/CANAAN-DOG-273.html
32,WETTERHOUN (221),/en/nomenclature/WETTERHOUN-221.html
10,DOGUE DE BORDEAUX (116),/en/nomenclature/DOGUE-DE-BORDEAUX-116.html


In [20]:
fci_breeds_df[["breed_orig", "breed_en"]] = fci_breeds_df["breed"].str.split(
    "(", n=1, expand=True
)
fci_breeds_df.sample(3)

,breed,link,breed_orig,breed_en
27,SUOMENLAPINKOIRA (189) (FINNISH LAPPONIAN DOG),/en/nomenclature/FINNISH-LAPPONIAN-DOG-189.html,SUOMENLAPINKOIRA,189) (FINNISH LAPPONIAN DOG)
22,POSAVSKI GONIC (154) (POSAVATZ HOUND),/en/nomenclature/POSAVATZ-HOUND-154.html,POSAVSKI GONIC,154) (POSAVATZ HOUND)
31,VÄSTGÖTASPETS (14) (SWEDISH VALLHUND),/en/nomenclature/SWEDISH-VALLHUND-14.html,VÄSTGÖTASPETS,14) (SWEDISH VALLHUND)


Turned out that webpage only had 33 breeds although it was from the all breeds link.

In [21]:
only_letters_pattern = r"\(?([A-Za-z-\.\s]+)\)"
fci_breeds_df["breed_en"] = fci_breeds_df["breed_en"].str.extract(only_letters_pattern)
fci_breeds_df["breed_en"] = fci_breeds_df["breed_en"].fillna(
    fci_breeds_df["breed_orig"].transform(lambda x: x)
)
fci_breeds_df["breed_orig"] = fci_breeds_df["breed_orig"].str.strip().str.lower()
fci_breeds_df["breed_en"] = fci_breeds_df["breed_en"].str.strip().str.lower()
# fci_breeds_df

In [22]:
fci_breeds_df["weblink"] = fci_breeds_df["link"].apply(lambda x: "www.fci.be" + x)
fci_breeds_df

,breed,link,breed_orig,breed_en,weblink
0,BERNER SENNENHUND (45) (BERNESE MOUNTAIN DOG),/en/nomenclature/BERNESE-MOUNTAIN-DOG-45.html,berner sennenhund,bernese mountain dog,www.fci.be/en/nomenclature/BERNESE-MOUNTAIN-DO...
1,BOLOGNESE (196),/en/nomenclature/BOLOGNESE-196.html,bolognese,bolognese,www.fci.be/en/nomenclature/BOLOGNESE-196.html
2,CANAAN DOG (273),/en/nomenclature/CANAAN-DOG-273.html,canaan dog,canaan dog,www.fci.be/en/nomenclature/CANAAN-DOG-273.html
3,CESKÝ TERIER (246) (CESKY TERRIER),/en/nomenclature/CESKY-TERRIER-246.html,ceský terier,cesky terrier,www.fci.be/en/nomenclature/CESKY-TERRIER-246.html
4,CHIHUAHUEÑO (218) (CHIHUAHUA),/en/nomenclature/CHIHUAHUA-218.html,chihuahueño,chihuahua,www.fci.be/en/nomenclature/CHIHUAHUA-218.html
5,CIMARRÓN URUGUAYO (353),/en/nomenclature/CIMARRON-URUGUAYO-353.html,cimarrón uruguayo,cimarrón uruguayo,www.fci.be/en/nomenclature/CIMARRON-URUGUAYO-3...
6,CIOBANESC ROMÂNESC DE BUCOVINA (357) (ROMANIAN...,/en/nomenclature/ROMANIAN-BUCOVINA-SHEPHERD-35...,ciobanesc românesc de bucovina,romanian bucovina shepherd,www.fci.be/en/nomenclature/ROMANIAN-BUCOVINA-S...
7,DALMATINSKI PAS (153) (DALMATIAN),/en/nomenclature/DALMATIAN-153.html,dalmatinski pas,dalmatian,www.fci.be/en/nomenclature/DALMATIAN-153.html
8,DANSK-SVENSK GÅRDSHUND (356) (DANISH-SWEDISH F...,/en/nomenclature/DANISH-SWEDISH-FARMDOG-356.html,dansk-svensk gårdshund,danish-swedish farmdog,www.fci.be/en/nomenclature/DANISH-SWEDISH-FARM...
9,DOGO ARGENTINO (292),/en/nomenclature/DOGO-ARGENTINO-292.html,dogo argentino,dogo argentino,www.fci.be/en/nomenclature/DOGO-ARGENTINO-292....


In [23]:
fci_breeds_df.to_csv("../data/fci_dog_breeds.csv", index=False)

In [24]:
akc_breeds_df.breed_group.unique().tolist()
breed_type_match = {}
fci_breed_groups = [
    "Sheepdogs and Cattledogs (except Swiss Cattledogs)",
    "Pinscher and Schnauzer - Molossoid and Swiss Mountain and Cattledogs",
    "Terriers",
    "Dachshunds",
    "Spitz and primitive types",
    "Scent hounds and related breeds",
    "Pointing Dogs",
    "Retrievers - Flushing Dogs Water Dogs",
    "Companion and Toy Dogs",
    "Sighthounds",
]
fci_breeds = {fci_breed_groups.index(group) + 1: group for group in fci_breed_groups}
akc_breed_groups = akc_breeds_df.breed_group.unique().tolist()

#### Spitz Breed Group

Get info from this story about the spitz breeds.

In [25]:
spitz_story_url = "https://www.akc.org/expert-advice/dog-breeds/spitz-dog-breeds/"

spitz_response = urlopen(spitz_story_url)
spitz_html_content = spitz_response.read()

In [26]:
spitz_soup = BeautifulSoup(spitz_html_content, "lxml")

# get the breeds linted in the 'tag-set__item-link' class
elements = spitz_soup.find_all("a", class_="tag-set__item-link")
spitz_breeds = [element.text for element in elements]

display(spitz_breeds)

# find these breeds in the akc_breeds_df
akc_breeds_df[akc_breeds_df.breed.isin(spitz_breeds)]

['pomeranian',
 'norwegian buhund',
 'norwich terrier',
 'alaskan malamute',
 'icelandic sheepdog',
 'swedish vallhund',
 'keeshond',
 'finnish lapphund',
 'akita',
 'shiba inu',
 'spitz breeds',
 'american eskimo dog',
 'chow chow',
 'norwegian elkhound',
 'siberian husky',
 'samoyed']

,breed_group,breed,links
18,herding group,finnish lapphund,https://www.akc.org/dog-breeds/finnish-lapphund/
20,herding group,icelandic sheepdog,https://www.akc.org/dog-breeds/icelandic-sheep...
24,herding group,norwegian buhund,https://www.akc.org/dog-breeds/norwegian-buhund/
33,herding group,swedish vallhund,https://www.akc.org/dog-breeds/swedish-vallhund/
53,hound group,norwegian elkhound,https://www.akc.org/dog-breeds/norwegian-elkho...
81,toy group,pomeranian,https://www.akc.org/dog-breeds/pomeranian/
89,non-sporting group,american eskimo dog,https://www.akc.org/dog-breeds/american-eskimo...
94,non-sporting group,chow chow,https://www.akc.org/dog-breeds/chow-chow/
99,non-sporting group,keeshond,https://www.akc.org/dog-breeds/keeshond/
105,non-sporting group,shiba inu,https://www.akc.org/dog-breeds/shiba-inu/


#### Year breed was recognized by AKC

In [27]:
akc_breed_year_url = "https://www.akc.org/press-center/articles-resources/facts-and-stats/breeds-year-recognized/"

akc_breed_year_response = urlopen(akc_breed_year_url)
akc_breed_year_html_content = akc_breed_year_response.read()

In [28]:
akc_breed_year_soup = BeautifulSoup(akc_breed_year_html_content, "html.parser")

In [29]:
# convert to etree
akc_breed_year_parsed_html = etree.HTML(akc_breed_year_html_content)

# get the elements with the tag span and that have '&nbsp' in the text
akc_breed_year_elements = akc_breed_year_parsed_html.xpath(
    "//*[contains(text(), '\u00A0')]"
)
[element.text.strip() for element in akc_breed_year_elements]

# get elements in the class 'content-body__text-long'
akc_breed_year_element = akc_breed_year_soup.find(
    "div", class_="content-body__text-long"
)

# find the p tag in each of those elements and get the text in the span tag in the p tag
year_breed_list = [
    element.text.strip().replace("\xa0", "")
    for element in akc_breed_year_element
    if element.text.strip()
][2:]

In [30]:
year_breed_df = pd.DataFrame(
    [yb.split("–") for yb in year_breed_list], columns=["year", "breed"]
)
for col in year_breed_df.columns:
    year_breed_df[col] = year_breed_df[col].str.strip()
year_breed_df["year"] = year_breed_df["year"].astype(int)
year_breed_df["breed"] = year_breed_df["breed"].str.lower()

# make breed the first column
year_breed_df = year_breed_df[["breed", "year"]]
year_breed_df

,breed,year
0,pointer,1878
1,retriever (chesapeake bay),1878
2,spaniel (clumber),1878
3,spaniel (cocker),1878
4,setter (english),1878
...,...,...
195,belgian laekenois,2020
196,biewer terrier,2021
197,bracco italiano,2022
198,mudi,2022


In [31]:
# akc_breeds_df

In [32]:
# define a function to match breed names using FuzzyWuzzy
def match_breed_name(name, choices):
    mismo, score, *_ = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    return mismo, score


# create a copy of the akc_breeds_df dataframe to avoid modifying the original
new_akc_df = akc_breeds_df.copy()
new_akc_df.sample(5)

,breed_group,breed,links
214,foundation stock service,alaskan klee kai,https://www.akc.org/dog-breeds/alaskan-klee-kai/
9,herding group,bergamasco,https://www.akc.org/dog-breeds/bergamasco-shee...
241,foundation stock service,hamiltonstovare,https://www.akc.org/dog-breeds/hamiltonstovare/
95,non-sporting group,coton de tulear,https://www.akc.org/dog-breeds/coton-de-tulear/
162,terrier group,russell terrier,https://www.akc.org/dog-breeds/russell-terrier/


In [33]:
# create new columns in the new_akc_df dataframe to store the closest matching breed name and score
new_akc_df["closest_match"], new_akc_df["match_score"] = zip(
    *new_akc_df["breed"].apply(lambda x: match_breed_name(x, year_breed_df["breed"]))
)
missed_breeds = set(year_breed_df["breed"].unique().tolist()) - set(
    new_akc_df["closest_match"].unique().tolist()
)
# fix the one that was missed
new_akc_df.loc[new_akc_df["breed"].str.contains("plott"), "closest_match"] = "plott"


# merged_df

In [34]:
pd.set_option("display.max_rows", 300)

new_akc_df.loc[
    new_akc_df["breed_group"].str.contains(
        r"miscellaneous class|foundation stock service"
    ),
    "closest_match",
] = ""

In [35]:
# merge the dataframes on the closest_match column
merged_df = pd.DataFrame()
merged_df = (
    new_akc_df.merge(
        year_breed_df, left_on="closest_match", right_on="breed", how="left"
    )
    .drop(["breed_y", "match_score", "closest_match"], axis=1)
    .rename(columns={"breed_x": "breed"})
)

In [36]:
merged_df.to_csv("../data/akc_dog_breeds.csv", index=False)
# merged_df.sort_values("breed")



#### AKC physical traits

In [ ]:


akc_links = akc_breeds_df["links"].tolist()

In [97]:
def get_breed_info(driver, link):
    breed_metadata = defaultdict(str)
    driver.get(link)

    try:
        breed = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h1.page-header__title"))
        )
        breed_metadata["breed"] = breed.text

        # temperment
        temperment = driver.find_element(
            By.CSS_SELECTOR, "p.breed-page__intro__temperment"
        )
        breed_metadata["temperment"] = temperment.text

        # height, weight, life expectancy
        elements = driver.find_elements(
            By.CSS_SELECTOR, "div.breed-page__hero__overview__icon-block"
        )
        for ele in elements:
            text = ele.find_element(By.TAG_NAME, "p").text
            height_match = re.search(r"(\d+)(?:\s*-\s*(\d+))?\s*inches?", text)
            weight_match = re.search(r"(\d+)(?:\s*-\s*(\d+))?\s*pounds?", text)
            life_expectancy_match = re.search(r"(\d+)(?:\s*-\s*(\d+))?\s*years?", text)

            if height_match:
                breed_metadata["height"] = height_match.group(0)
            elif weight_match:
                breed_metadata["weight"] = weight_match.group(0)
            elif life_expectancy_match:
                breed_metadata["life_expectancy"] = life_expectancy_match.group(0)

    except NoSuchElementException as e:
        print("No such element", e)

    return breed_metadata

In [98]:
my_d = start_driver()
breed_data_driver = partial(get_breed_info, my_d)

list_of_dicts = []
for link in tqdm(akc_links):
    list_of_dicts.append(breed_data_driver(link))
my_d.quit()


100%|██████████| 280/280 [09:34<00:00,  2.05s/it]


,breed,temperment,height,weight,life_expectancy
0,Australian Cattle Dog,Loyal / Intelligent / Tenacious,18-20 inches,35-50 pounds,12-16 years
1,Australian Shepherd,smart / work-oriented / exuberant,20-23 inches,50-65 pounds,12-15 years
2,Bearded Collie,smart / bouncy / charismatic,21-22 inches,45-55 pounds,12-14 years
3,Beauceron,faithful / gentle / obedient,5 inches,70-110 pounds,10-12 years
4,Belgian Laekenois,alert / intelligent / affectionate,24-26 inches,55-65 pounds,10-12 years
5,Belgian Malinois,confident / smart / hardworking,24-26 inches,60-80 pounds,14-16 years
6,Belgian Sheepdog,bright / watchful / serious-minded,24-26 inches,55-75 pounds,12-14 years
7,Belgian Tervuren,alert / intelligent / courageous,24-26 inches,55-75 pounds,12-14 years
8,Bergamasco Sheepdog,independent / intelligent / sociable,5 inches,70-84 pounds,13-15 years
9,Berger Picard,loyal / good-natured / observant,5 inches,50-70 pounds,12-13 years


In [104]:
akc_physical_traits = pd.DataFrame(list_of_dicts)
akc_physical_traits["breed"] = akc_physical_traits["breed"].str.lower()
merger_df = merged_df.merge(akc_physical_traits, on="breed", how="left")

,breed_group,breed,links,year,temperment,height,weight,life_expectancy
0,herding group,australian cattle dog,https://www.akc.org/dog-breeds/australian-catt...,1980.0,Loyal / Intelligent / Tenacious,18-20 inches,35-50 pounds,12-16 years
1,herding group,australian shepherd,https://www.akc.org/dog-breeds/australian-shep...,1991.0,smart / work-oriented / exuberant,20-23 inches,50-65 pounds,12-15 years
2,herding group,bearded collie,https://www.akc.org/dog-breeds/bearded-collie/,1976.0,smart / bouncy / charismatic,21-22 inches,45-55 pounds,12-14 years
3,herding group,beauceron,https://www.akc.org/dog-breeds/beauceron/,2007.0,faithful / gentle / obedient,5 inches,70-110 pounds,10-12 years
4,herding group,belgian laekenois,https://www.akc.org/dog-breeds/belgian-laekenois/,2020.0,alert / intelligent / affectionate,24-26 inches,55-65 pounds,10-12 years
5,herding group,belgian malinois,https://www.akc.org/dog-breeds/belgian-malinois/,1959.0,confident / smart / hardworking,24-26 inches,60-80 pounds,14-16 years
6,herding group,belgian sheepdog,https://www.akc.org/dog-breeds/belgian-sheepdog/,1912.0,bright / watchful / serious-minded,24-26 inches,55-75 pounds,12-14 years
7,herding group,belgian tervuren,https://www.akc.org/dog-breeds/belgian-tervuren/,1959.0,alert / intelligent / courageous,24-26 inches,55-75 pounds,12-14 years
8,herding group,bergamasco,https://www.akc.org/dog-breeds/bergamasco-shee...,2015.0,NaN,NaN,NaN,NaN
9,herding group,berger picard,https://www.akc.org/dog-breeds/berger-picard/,2015.0,loyal / good-natured / observant,5 inches,50-70 pounds,12-13 years


In [106]:
# akc_breeds_df
akc_physical_traits.sort_values("breed")

,breed,temperment,height,weight,life_expectancy
65,affenpinscher,confident / famously funny / fearless,5 inches,7-10 pounds,12-15 years
33,afghan hound,independent / sweet,25-27 inches,50-60 pounds,12-18 years
141,airedale terrier,friendly / courageous / clever,23 inches,50-70 pounds,11-14 years
171,akita,dignified / courageous / profoundly loyal,26-28 inches,100-130 pounds,10-14 years
213,alaskan klee kai,loyal / intelligent / vigilant,12 - 17 inches,NaN,13-16 years
172,alaskan malamute,loyal / playful / affectionate,25 inches,85 pounds,10-14 years
214,american bulldog,loyal / self-confident,22-25 inches,75-100 pounds,10-12 years
34,american english coonhound,sociable / sweet / mellow,24-26 inches,45-65 pounds,11-12 years
88,american eskimo dog,Loyal / Smart / Versatile,9-12 inches,6-10 pounds,13-15 years
35,american foxhound,independent / easy-going / sweet-tempered,22-25 inches,65-70 pounds,11-13 years
